In [1]:
import paho.mqtt.publish as publish
from core.settings import settings
import numpy as np
from core.simulation.modelica import ModelicaAgent
from core.forecasts.buildingEnergyForecast import BuildingEnergyForecast
import time
from pathlib import Path
from core.utils.fiware_utils import clean_up
import json
from core.data_models.attribute import Attribute
import copy
import pandas as pd

%load_ext autoreload
%autoreload 2

In [40]:
publish.single(topic='/mpc',
               hostname=settings.MQTT_HOST,
               port=settings.MQTT_PORT)

In [49]:
schema_path = Path.cwd() / 'core' / 'data_models' /\
        'schema' / 'BuildingEnergyForecast.json'
with open(schema_path) as f:
    data_model = json.load(f)

entity_id = f'BuildingEnergyForecast:DEQ:MVP:test'
building_energy_forecast = BuildingEnergyForecast(
    entity_id=entity_id,
    entity_type="BuildingEnergyForecast",
    building_ix=2,
    data_model=copy.deepcopy(data_model)  # necessary because id gets popped from data_model. TODO change that in json schema to fiware converter
)


27.02.2024-09:59:01 INFO BuildingEnergyForecast:DEQ:MVP:test: Heat Predictor learned


In [50]:
y_hat, input_tsd = building_energy_forecast.predict()

print(y_hat)
display(input_tsd)

Variables,heating_2//1,heating_2//2,heating_2//3,sin,sin//1,sin//2,sin//3,ones
Tags,raw,raw,raw,raw,raw,raw,raw,raw
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


27.02.2024-09:59:02 WARNING BuildingEnergyForecast:DEQ:MVP:test: NaNs in array. Filling with 0
27.02.2024-09:59:02 INFO BuildingEnergyForecast:DEQ:MVP:test: Push successfull


[('sin', 'raw'), ('heating_2//1', 'raw'), ('ones', 'raw')]


TypeError: cannot unpack non-iterable NoneType object

In [8]:
building_energy_forecast.predictor.data_shifter(tsd)

ValueError: Length of values (3) does not match length of index (1)

In [28]:
def data_shifter(data):

    shift_dict = {}

    for col in list(data):
        _col, tag = col
        
        if '//' in _col:
            name, shift = _col.split('//')
            
        else:
            name = _col
            shift = 0
        if any(i in name for i in ['sin', 'ones']):
            continue
        
        if name not in shift_dict:
            shift_dict[name] = []
            
        shift_dict[name].append((col, int(shift)))
        
    for key, val in shift_dict.items():
        shift_dict[key] = sorted(val, key=lambda x: x[1])
        
    for name, shift_vars in shift_dict.items():
        print(name, shift_vars)
        use_shift_vars = [i[0] for i in shift_vars]
        this_data = data.loc[0, use_shift_vars].to_numpy()
        n = this_data.shape[0]
        matrix = np.full((n, n), np.nan)
        
        for i in range(n):
            matrix[i, i:n] = this_data[0: n-i]

        
        display(data)
        
        data.loc[0: n-1, use_shift_vars] = matrix

    return data

In [31]:
tsd_use = tsd.copy()

def sin_extension(tsd,
                  n_horizon):
    
    def apply_sin_calc(x):
        b = 2 * np.pi / 24
        c = -np.pi / 2
        return np.sin(b * x + c)
    print(tsd.shape)
    assert (tsd.shape[0] == 1)
    
    new_rows = pd.DataFrame(np.nan, index=range(n_horizon - 1), columns=tsd.columns)
    df = pd.concat([tsd, new_rows], ignore_index=True)
    sin_cols = [i for i in list(df) if 'sin' in i[0]]

    for i in range(1, len(df)):
        df.loc[df.index[i], sin_cols] = (df.loc[df.index[i-1], sin_cols] + 1) % 24
    
    
    df[sin_cols] = df[sin_cols].apply(apply_sin_calc)
    if ('ones', 'raw') in list(df):
        df[('ones', 'raw')] = 1
        
    return df

tsd_use = sin_extension(tsd, 
                         n_horizon=12)
data_shifter(tsd_use)

(1, 7)
heating_0 [(('heating_0//1', 'raw'), 1), (('heating_0//2', 'raw'), 2), (('heating_0//3', 'raw'), 3)]


Variables,heating_0//1,heating_0//2,heating_0//3,sin//1,sin//2,sin//3,ones
Tags,raw,raw,raw,raw,raw,raw,raw
0,2567.81,2345.05,2083.02,-0.500000,-0.707107,-0.866025,1
1,NaN,NaN,NaN,-0.258819,-0.500000,-0.707107,1
2,NaN,NaN,NaN,0.000000,-0.258819,-0.500000,1
3,NaN,NaN,NaN,0.258819,0.000000,-0.258819,1
4,NaN,NaN,NaN,0.500000,0.258819,0.000000,1
5,NaN,NaN,NaN,0.707107,0.500000,0.258819,1
6,NaN,NaN,NaN,0.866025,0.707107,0.500000,1
7,NaN,NaN,NaN,0.965926,0.866025,0.707107,1
8,NaN,NaN,NaN,1.000000,0.965926,0.866025,1


Variables,heating_0//1,heating_0//2,heating_0//3,sin//1,sin//2,sin//3,ones
Tags,raw,raw,raw,raw,raw,raw,raw
0,2567.81,2345.05,2083.02,-0.500000,-0.707107,-0.866025,1
1,NaN,2567.81,2345.05,-0.258819,-0.500000,-0.707107,1
2,NaN,NaN,2567.81,0.000000,-0.258819,-0.500000,1
3,NaN,NaN,NaN,0.258819,0.000000,-0.258819,1
4,NaN,NaN,NaN,0.500000,0.258819,0.000000,1
5,NaN,NaN,NaN,0.707107,0.500000,0.258819,1
6,NaN,NaN,NaN,0.866025,0.707107,0.500000,1
7,NaN,NaN,NaN,0.965926,0.866025,0.707107,1
8,NaN,NaN,NaN,1.000000,0.965926,0.866025,1


Variables,heating_0//1,heating_0//2,heating_0//3,sin//1,sin//2,sin//3,ones
Tags,raw,raw,raw,raw,raw,raw,raw
0,2567.81,2345.05,2083.02,4.0,3.0,2.0,1.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df

Variables,heating_0//1,heating_0//2,heating_0//3,sin//1,sin//2,sin//3,ones
Tags,raw,raw,raw,raw,raw,raw,raw
0,2567.81,2345.05,2083.02,4.0,3.0,2.0,1.0
1,NaN,NaN,NaN,5.0,4.0,3.0,NaN
2,NaN,NaN,NaN,6.0,5.0,4.0,NaN
3,NaN,NaN,NaN,7.0,6.0,5.0,NaN
4,NaN,NaN,NaN,8.0,7.0,6.0,NaN
5,NaN,NaN,NaN,9.0,8.0,7.0,NaN
6,NaN,NaN,NaN,10.0,9.0,8.0,NaN
7,NaN,NaN,NaN,11.0,10.0,9.0,NaN
8,NaN,NaN,NaN,12.0,11.0,10.0,NaN


In [15]:
df = data_shifter(df)
df

Variables,heating_0//1,heating_0//2,heating_0//3,sin//1,sin//2,sin//3,ones
Tags,raw,raw,raw,raw,raw,raw,raw
0,2567.81,2345.05,2083.02,4.0,3.0,2.0,1.0
1,NaN,2567.81,2345.05,NaN,4.0,3.0,NaN
2,NaN,NaN,2567.81,NaN,NaN,4.0,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
